In [1]:
%cd /home/giorgian/Documente/Cercetare/Fairness/nn_fairness

/home/giorgian/Documente/Cercetare/Fairness/nn_fairness


In [2]:
import nn_fairness as nnf
import json
import numpy as np
from prob import ProbabilityDensityComputer
import pickle

/home/giorgian/.local/share/venvs/fairness/lib/python3.11/site-packages/skl2onnx/algebra/onnx_ops.py:159: UserWarning: OpSchema.FormalParameter.typeStr is deprecated and will be removed in 1.16. Use OpSchema.FormalParameter.type_str instead.
  tys = obj.typeStr or ''
/home/giorgian/.local/share/venvs/fairness/lib/python3.11/site-packages/skl2onnx/algebra/automation.py:154: UserWarning: OpSchema.FormalParameter.isHomogeneous is deprecated and will be removed in 1.16. Use OpSchema.FormalParameter.is_homogeneous instead.
  if getattr(obj, 'isHomogeneous', False):
/usr/lib/python3.11/site-packages/jinja2/environment.py:485: UserWarning: OpSchema.FormalParameter.typeStr is deprecated and will be removed in 1.16. Use OpSchema.FormalParameter.type_str instead.
  return getattr(obj, attribute)


In [3]:
nnf.set_settings()

In [4]:
with open('configs/config-compas.json') as handle:
    config = json.load(handle)

In [5]:
with open(config['train_data_path'], 'rb') as f:
    data_dict = pickle.load(f)
    X = data_dict['X_train']
    y = data_dict['y_train']
    
c1 = config['class_1']
c2 = config['class_2']
class_1_indices = np.array(c1['indices'])
class_1_values = np.array(c1['values'])
class_2_indices = np.array(c2['indices'])
class_2_values = np.array(c2['values'])
def is_class_1(x):
    return np.allclose(x[class_1_indices], class_1_values, atol=1e-1)

def is_class_2(x):
    return np.allclose(x[class_2_indices], class_2_values, atol=1e-1)

class_1_prob = ProbabilityDensityComputer(
        X,
        config['discrete_indices'],
        config['continuous_indices'],
        config['one_hot_indices'],
        config['fixed_indices'],
        is_class_1
)

class_2_prob = ProbabilityDensityComputer(
        X,
        config['discrete_indices'],
        config['continuous_indices'],
        config['one_hot_indices'],
        config['fixed_indices'],
        is_class_2
)

In [6]:
input_config = nnf.InputConfig(config['one_hot_indices'], config['discrete_indices'])

In [14]:
fc = nnf.FairnessCalculator.from_onnx_file(config['models'][1][1], input_config)

In [15]:
c1 = config['class_1']
c2 = config['class_2']
class_1_box = np.array(c1['box'], dtype=np.float32)
class_2_box = np.array(c2['box'], dtype=np.float32)

In [16]:
class_1_box

array([[0., 1.],
       [0., 1.],
       [0., 0.],
       [0., 1.]], dtype=float32)

In [17]:
res = fc.forward({'white': class_1_box, 'black': class_2_box})

In [18]:
res.advantage({'white': class_1_prob, 'black': class_2_prob}, label='safe')

{('white', 'black'): (0.1340127716765126,
  0.6662315740639089,
  '13.40127716765126% of all white individuals would have not been classified safe if only they had been black'),
 ('black', 'white'): (-1.1102230246251565e-16,
  0.306423983636468,
  '-1.1102230246251565e-14% of all black individuals would have not been classified safe if only they had been white')}

In [19]:
res.disadvantage({'white': class_1_prob, 'black': class_2_prob}, label='safe')

{('black', 'white'): (0.13655877933685334,
  0.44298276297332145,
  '13.655877933685334% of all black individuals would have been classified safe if only they had been white'),
 ('white', 'black'): (4.440892098500626e-16,
  0.5322188023873966,
  '4.440892098500626e-14% of all white individuals would have been classified safe if only they had been black')}

In [20]:
res.preference({'white': class_1_prob, 'black': class_2_prob}, label='safe')

{('white', 'black'): (0.306423983636468,
  0.6662315740639089,
  'black individuals are considered safe at 45.99361476780955% of the rate of white individuals'),
 ('black', 'white'): (0.6662315740639089,
  0.306423983636468,
  'white individuals are considered safe at 217.42148449264516% of the rate of black individuals')}